In [ ]:
!pip install pytube
!pip install opencv-python
!pip install ImageHash
!pip install fpdf
!pip install img2pdf

In [ ]:
import cv2
import os
import tempfile
from pytube import YouTube
from PIL import Image
import img2pdf
from imagehash import phash

def download_youtube_video(url):
    yt = YouTube(url)
    stream = yt.streams.filter(file_extension='mp4').get_highest_resolution()
    if stream is None:
        print("No mp4 stream found")
        return None
    temp_video_file = tempfile.NamedTemporaryFile(delete=False)
    temp_video_file.close()
    stream.download(filename=temp_video_file.name)
    return temp_video_file.name

def process_frames(video_file):
    capture = cv2.VideoCapture(video_file)
    frame_count = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(capture.get(cv2.CAP_PROP_FPS))
    
    frames = []
    for i in range(0, frame_count, fps):
        capture.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = capture.read()
        if ret:
            frames.append(frame)
    capture.release()
    return frames

def detect_slide_changes(frames):
    slide_frames = [frames[0]]
    last_frame_hash = phash(Image.fromarray(cv2.cvtColor(frames[0], cv2.COLOR_BGR2RGB)))
    for frame in frames[1:]:
        current_frame_hash = phash(Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))
        if current_frame_hash - last_frame_hash > 10:
            slide_frames.append(frame)
        last_frame_hash = current_frame_hash
    return slide_frames

def save_screenshots(slide_frames, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    screenshot_paths = []
    for i, slide_frame in enumerate(slide_frames):
        save_path = os.path.join(output_folder, f'slide_{i+1}.png')
        cv2.imwrite(save_path, slide_frame)
        screenshot_paths.append(save_path)
    return screenshot_paths

def convert_screenshots_to_pdf(screenshot_paths, output_pdf_path):
    with open(output_pdf_path, "wb") as f:
        f.write(img2pdf.convert(screenshot_paths))

def main():
    youtube_video_url = input("Enter YouTube video URL: ")
    output_folder = input("Enter output folder path: ")

    video_file = download_youtube_video(youtube_video_url)
    if video_file is None:
        print("Failed to download video")
        return
    print("Video downloaded")
    frames = process_frames(video_file)
    print("Frames processed")
    slide_frames = detect_slide_changes(frames)
    print("Slide changes detected")
    screenshot_paths = save_screenshots(slide_frames, output_folder)
    print("Screenshots saved")
    output_pdf_path = os.path.join(output_folder, "slides.pdf")
    convert_screenshots_to_pdf(screenshot_paths, output_pdf_path)
    print("Screenshots converted and merged into PDF")
    os.remove(video_file)

if __name__ == '__main__':
    main()

Enter YouTube video URL: https://www.youtube.com/watch?v=lLo09ZKA0ZM
Enter output folder path: shots
Video downloaded
